<a href="https://colab.research.google.com/github/monimiller/womens_wc_23/blob/main/Monica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 3.8 MB/s eta 0:00:00


In [1]:
# Basics
import pandas as pd
import numpy as np
# Viz
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
sns.set()
#sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, f1_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

Load data

In [3]:
df = pd.read_csv('/content/match_history.csv')

# New Section

In [4]:
df

,date,home_team,away_team,home_score,away_score
0,2003-09-24,Norway,Brazil,1,4
1,2003-09-25,China PR,Australia,1,1
2,2003-09-27,Switzerland,Italy,0,1
3,2003-09-27,Canada,Japan,3,1
4,2003-09-27,Argentina,Germany,1,6
...,...,...,...,...,...
1145,2023-07-07,Germany,Zambia,2,3
1146,2023-07-13,Vietnam,Spain,0,9
1147,2023-07-14,Australia,France,1,0
1148,2023-07-14,Costa Rica,South Africa,0,2


In [5]:
from datetime import datetime
# Correcting data types
col_types = {
    'home_score' : 'int',
    'away_score' : 'int'
}

# Change col types
df = df.astype(col_types)

# Change Date to Datetime
df['date'] = pd.to_datetime(df['date'])

# Check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1150 entries, 0 to 1149
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        1150 non-null   datetime64[ns]
 1   home_team   1150 non-null   object        
 2   away_team   1150 non-null   object        
 3   home_score  1150 non-null   int64         
 4   away_score  1150 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 45.0+ KB


In [6]:
# Creating the result column with -1 (home team lost), 0 (draw), 1 (home team won)
conditions = [ df.home_score > df.away_score,
               df.home_score == df.away_score,
              df.home_score < df.away_score ]
vals = [1,0,-1]

df['result'] = np.select(conditions, vals)

In [7]:
df

,date,home_team,away_team,home_score,away_score,result
0,2003-09-24,Norway,Brazil,1,4,-1
1,2003-09-25,China PR,Australia,1,1,0
2,2003-09-27,Switzerland,Italy,0,1,-1
3,2003-09-27,Canada,Japan,3,1,1
4,2003-09-27,Argentina,Germany,1,6,-1
...,...,...,...,...,...,...
1145,2023-07-07,Germany,Zambia,2,3,-1
1146,2023-07-13,Vietnam,Spain,0,9,-1
1147,2023-07-14,Australia,France,1,0,1
1148,2023-07-14,Costa Rica,South Africa,0,2,-1


In [8]:
# Creating datasets for performance of the teams at home games
home_victories = df.query('result==1').groupby('home_team')['result'].count().sort_values(ascending=False).reset_index()
home_losses = df.query('result==-1').groupby('home_team')['result'].count().sort_values(ascending=False).reset_index()
home_gf = df.groupby('home_team').home_score.sum().sort_values(ascending=False).reset_index()
home_ga = df.groupby('home_team').away_score.sum().sort_values(ascending=False).reset_index()
home_avg_goals = df.groupby('home_team').home_score.mean()

# Creating datasets for performance of the teams at aways games
away_victories = df.query('result==-1').groupby('away_team')['result'].count().sort_values(ascending=False).reset_index()
away_losses = df.query('result==1').groupby('away_team')['result'].count().sort_values(ascending=False).reset_index()
away_gf = df.groupby('away_team').away_score.sum().sort_values(ascending=False).reset_index()
away_ga = df.groupby('away_team').home_score.sum().sort_values(ascending=False).reset_index()
away_avg_goals = df.groupby('away_team').away_score.mean()

# Qty games played
games = {team:df.query('away_team == @team | home_team == @team')['result'].count() for team in df.home_team.unique()}
games = pd.DataFrame.from_dict(games, orient='index').reset_index().rename(columns={'index': 'team', 0:'games'})

,home_team,home_score
0,United States,508
1,Germany,186
2,Brazil,138
3,Sweden,104
4,China PR,94
5,France,90
6,England,88
7,Canada,84
8,Japan,78
9,Norway,68


In [9]:
# Creating performance measurements wins and losses
teams_performance = pd.DataFrame({'team': df.home_team.unique()})

# Games Played
teams_performance = (
    teams_performance
    .merge(games, on='team',how='left')
    .fillna(0))

# Add home wins
teams_performance = (
    teams_performance
    .merge(home_victories, left_on='team', right_on='home_team', how='left')
    .fillna(0)
    .drop('home_team', axis=1)
    .rename(columns={'result':'home_wins'}) )

# Add away wins
teams_performance = (
    teams_performance
    .merge(away_victories, left_on='team', right_on='away_team', how='left')
    .fillna(0)
    .drop('away_team', axis=1)
    .rename(columns={'result':'away_wins'}) )

# Add home losses
teams_performance = (
    teams_performance
    .merge(home_losses, left_on='team', right_on='home_team', how='left')
    .fillna(0)
    .drop('home_team', axis=1)
    .rename(columns={'result':'home_losses'}) )

# Add away losses
teams_performance = (
    teams_performance
    .merge(away_losses, left_on='team', right_on='away_team', how='left')
    .fillna(0)
    .drop('away_team', axis=1)
    .rename(columns={'result':'away_losses'}) )

# Add home GF
teams_performance = (
    teams_performance
    .merge(home_gf, left_on='team', right_on='home_team', how='left')
    .fillna(0)
    .drop('home_team', axis=1)
    .rename(columns= {'home_score':'home_gf'}) )

# Add home GA
teams_performance = (
    teams_performance
    .merge(home_ga, left_on='team', right_on='home_team', how='left')
    .fillna(0)
    .drop('home_team', axis=1)
    .rename(columns= {'away_score':'home_ga'}) )

# Add Away GF
teams_performance = (
    teams_performance
    .merge(away_gf, left_on='team', right_on='away_team', how='left')
    .fillna(0)
    .drop('away_team', axis=1)
    .rename(columns= {'away_score':'away_gf'}) )

# Add Away GA
teams_performance = (
    teams_performance
    .merge(away_ga, left_on='team', right_on='away_team', how='left')
    .fillna(0)
    .drop('away_team', axis=1)
    .rename(columns= {'home_score':'away_ga'}) )

# Add Overal performance = [Home Wins + Away Wins]/[Games Played]
teams_performance['performance'] = (
    teams_performance['home_wins'].add(teams_performance['away_wins']).div(teams_performance['games']))

# Teams Performance Dataset
teams_performance.sort_values(by='performance', ascending=False).head()

,team,games,home_wins,away_wins,home_losses,away_losses,home_gf,home_ga,away_gf,away_ga,performance
32,Venezuela,1,1.0,0.0,0.0,0.0,4,1,0.0,0.0,1.000000
6,United States,254,145.0,53.0,7.0,11.0,508,99,144.0,60.0,0.779528
8,Germany,133,54.0,30.0,17.0,13.0,186,61,105.0,42.0,0.631579
13,France,102,32.0,20.0,11.0,18.0,90,51,74.0,63.0,0.509804
5,Brazil,108,40.0,15.0,12.0,20.0,138,58,56.0,59.0,0.509259


In [10]:
teams_performance

,team,games,home_wins,away_wins,home_losses,away_losses,home_gf,home_ga,away_gf,away_ga,performance
0,Norway,111,20.0,19.0,17.0,36.0,68,62,68.0,108.0,0.351351
1,China PR,143,28.0,12.0,31.0,37.0,94,91,47.0,95.0,0.279720
2,Switzerland,55,5.0,4.0,10.0,23.0,24,39,32.0,84.0,0.163636
3,Canada,126,31.0,26.0,15.0,36.0,84,51,81.0,96.0,0.452381
4,Argentina,38,3.0,3.0,7.0,22.0,14,23,10.0,90.0,0.157895
5,Brazil,108,40.0,15.0,12.0,20.0,138,58,56.0,59.0,0.509259
6,United States,254,145.0,53.0,7.0,11.0,508,99,144.0,60.0,0.779528
7,Sweden,132,32.0,21.0,27.0,21.0,104,96,69.0,66.0,0.401515
8,Germany,133,54.0,30.0,17.0,13.0,186,61,105.0,42.0,0.631579
9,Denmark,105,16.0,24.0,20.0,27.0,45,49,80.0,94.0,0.380952


In [11]:
df

,date,home_team,away_team,home_score,away_score,result
0,2003-09-24,Norway,Brazil,1,4,-1
1,2003-09-25,China PR,Australia,1,1,0
2,2003-09-27,Switzerland,Italy,0,1,-1
3,2003-09-27,Canada,Japan,3,1,1
4,2003-09-27,Argentina,Germany,1,6,-1
...,...,...,...,...,...,...
1145,2023-07-07,Germany,Zambia,2,3,-1
1146,2023-07-13,Vietnam,Spain,0,9,-1
1147,2023-07-14,Australia,France,1,0,1
1148,2023-07-14,Costa Rica,South Africa,0,2,-1


In [12]:
df['year'] = df['date'].dt.year

In [13]:
df

,date,home_team,away_team,home_score,away_score,result,year
0,2003-09-24,Norway,Brazil,1,4,-1,2003
1,2003-09-25,China PR,Australia,1,1,0,2003
2,2003-09-27,Switzerland,Italy,0,1,-1,2003
3,2003-09-27,Canada,Japan,3,1,1,2003
4,2003-09-27,Argentina,Germany,1,6,-1,2003
...,...,...,...,...,...,...,...
1145,2023-07-07,Germany,Zambia,2,3,-1,2023
1146,2023-07-13,Vietnam,Spain,0,9,-1,2023
1147,2023-07-14,Australia,France,1,0,1,2023
1148,2023-07-14,Costa Rica,South Africa,0,2,-1,2023


In [14]:
test_index = df.query('year == 2012').index.min()

In [15]:
df_input = df[['home_team', 'away_team', 'result', 'home_score', 'away_score']]
df_input = (
    df_input
    .merge(teams_performance, left_on= 'home_team', right_on='team', how='left').fillna(0)
    .merge(teams_performance, left_on= 'away_team', right_on='team', suffixes=['_home','_away'], how='left').fillna(0) )
df_input.drop(['team_home', 'team_away'], axis=1, inplace=True)

In [16]:
train = df_input.iloc[:test_index,:]
test = df_input.iloc[test_index:,:]

In [17]:
# Categorical Features
"CatBoost needs to know which features are categorical"
cat_features = df_input.select_dtypes(include='object').columns.values
#cat_features = [df_input.columns.get_loc(c) for c in cat_features]

X = train.drop(['home_score','away_score','result'], axis=1)
y = train.result
Xt = test.drop(['home_score','away_score','result'], axis=1)
yt = test.result

In [18]:
X.columns

Index(['home_team', 'away_team', 'games_home', 'home_wins_home',
       'away_wins_home', 'home_losses_home', 'away_losses_home',
       'home_gf_home', 'home_ga_home', 'away_gf_home', 'away_ga_home',
       'performance_home', 'games_away', 'home_wins_away', 'away_wins_away',
       'home_losses_away', 'away_losses_away', 'home_gf_away', 'home_ga_away',
       'away_gf_away', 'away_ga_away', 'performance_away'],
      dtype='object')

In [59]:
# Catboost
model = CatBoostClassifier(iterations=10000, learning_rate=0.55)
model.fit(X, y, cat_features= cat_features, verbose=2000)

0:	learn: 0.9648124	total: 62.5ms	remaining: 10m 24s
2000:	learn: 0.0032238	total: 7.4s	remaining: 29.6s
4000:	learn: 0.0015143	total: 16.5s	remaining: 24.7s
6000:	learn: 0.0009994	total: 25.7s	remaining: 17.1s
8000:	learn: 0.0007441	total: 32.5s	remaining: 8.11s
9999:	learn: 0.0005921	total: 41.5s	remaining: 0us


In [60]:
pred_train = model.predict(X)
pred_test = model.predict(Xt)
# Test Score
f1_score(yt, pred_test, average=None)

array([0.55696203, 0.14457831, 0.72336449])

In [61]:
# Confusion Matrix Training
pd.DataFrame(confusion_matrix(y_true=y, y_pred=pred_train),
             index=['away', 'draw', 'home'],
             columns=['away', 'draw', 'home'])

,away,draw,home
away,47,2,28
draw,13,12,10
home,15,6,81


In [62]:
# Confusion Matrix Test
pd.DataFrame(confusion_matrix(y_true=yt, y_pred=pred_test),
             index=['away', 'draw', 'home'],
             columns=['away', 'draw', 'home'])

,away,draw,home
away,154,32,107
draw,49,18,99
home,57,33,387


In [84]:
match = pd.DataFrame({
                      'home_team': ['New Zealand', 'Philippines', 'New Zealand', 'Switzerland'],
                      'away_team': ['Norway', 'Switzerland', 'Philippines', 'Norway']
                      })

In [67]:
def encode_variables(df):
  "Input a dataframe and return a dataframe whith categorical variables encoded numericaly"
  # Find categorical vars
  cat_vars = df.select_dtypes(include=['object', 'category']).columns
  # Loop and transform
  for variable in cat_vars:
    df[variable] = df[variable].factorize()[0]

  return df

In [85]:
X_input = (
    match
    .merge(teams_performance, left_on= 'home_team', right_on='team', how='left').fillna(0)
    .merge(teams_performance, left_on= 'away_team', right_on='team', suffixes=['_home','_away'], how='left').fillna(0) )
X_input.drop(['team_home', 'team_away'], axis=1, inplace=True)

X_input = encode_variables(X_input)

In [86]:
# Results with CatBoost
model.predict_proba(X_input)

array([[7.82253807e-02, 3.33836082e-03, 9.18436259e-01],
       [7.91162703e-01, 1.51491305e-03, 2.07322384e-01],
       [2.48787185e-05, 1.82950286e-05, 9.99956826e-01],
       [9.05361145e-01, 4.30979076e-03, 9.03290643e-02]])

Random


In [87]:
# Create a copy of X
X_rf = X.copy()
Xt_rf = Xt.copy()

In [88]:
# Encode categorical variables
X_rf = encode_variables(X_rf)

# Encode categorical variables
Xt_rf = encode_variables(Xt_rf)

In [89]:
x_scaled = StandardScaler().fit_transform(X_rf)
dummy = RandomForestClassifier().fit(x_scaled, y)
dummy_importances = pd.DataFrame({'variables':X_rf.columns,
                                  'importance': dummy.feature_importances_}).sort_values(by='importance', ascending=False)
px.bar(dummy_importances,
       x= 'variables', y='importance',
       height=500, width=1000)

In [90]:
sc = []
for r in range(15,100):
  steps = [
    ('scale', StandardScaler()),
    ('model', RandomForestClassifier(random_state=r))
]

  # Final Pipeline
  model_rf = Pipeline( steps )
# Fit
  model_rf.fit(X_rf, y)
  m = model_rf.score(Xt_rf,yt)
  sc.append((r, m))


In [91]:
# Steps
steps = [
    ('scale', StandardScaler()),
    ('model', RandomForestClassifier(random_state=79))
]

# Final Pipeline
model_rf = Pipeline( steps )
# Fit
model_rf.fit(X_rf, y)

Pipeline(steps=[('scale', StandardScaler()),
                ('model', RandomForestClassifier(random_state=79))])

In [92]:
# Model score
print(model_rf.score(X_rf,y))
print(model_rf.score(Xt_rf,yt))

0.8598130841121495
0.5854700854700855


In [93]:
# Model predictions
pred_rf_test = model_rf.predict(Xt_rf)

In [94]:
# Test confusion matrix
pd.DataFrame(confusion_matrix(y_true=yt, y_pred=pred_rf_test),
             index=['away', 'draw', 'home'],
             columns=['away', 'draw', 'home'])

,away,draw,home
away,149,28,116
draw,45,21,100
home,68,31,378


In [95]:
# F1 score
f1_score(y_true=yt, y_pred=pred_rf_test, average=None)

array([0.53693694, 0.17073171, 0.70588235])

In [96]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import VotingClassifier

In [97]:
# Results with Random Forest
model_rf.predict_proba(X_input)

array([[0.57783333, 0.116     , 0.30616667],
       [0.60166667, 0.14583333, 0.2525    ],
       [0.14333333, 0.08166667, 0.775     ],
       [0.76666667, 0.11833333, 0.115     ]])

In [98]:
# Results with Random Forest
preds = model_rf.predict_proba(X_input)

In [99]:
# Predictions of the results
match['away'] = pd.DataFrame(preds)[0]
match['draw'] = pd.DataFrame(preds)[1]
match['home'] = pd.DataFrame(preds)[2]

,home_team,away_team,away,draw,home
0,New Zealand,Norway,0.577833,0.116000,0.306167
1,Philippines,Switzerland,0.601667,0.145833,0.252500
2,New Zealand,Philippines,0.143333,0.081667,0.775000
3,Switzerland,Norway,0.766667,0.118333,0.115000
